In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

from plotly import tools
from plotly.offline import init_notebook_mode, iplot
import plotly.plotly as py
import plotly.graph_objs as go

# output to notebook
init_notebook_mode(True)

In [14]:
size = 1200
x = np.arange(size)
com1 = np.random.rand(size)*.5
com2 = np.log(np.linspace(1,2,size))
com3 = np.array([np.cos(2*np.pi*np.arange(12)/12)*.5 for i in range(100)]).reshape(size,)
ts = com1+com2+com3

trace = go.Scatter(
    x=x,
    y=ts
)

fig = [trace]
iplot(fig)

## Double Exponential Smoothing

In [119]:
@jit
def des_nb(series, a, b, pred_interval):
    result = []
    for n in range(len(series)):
        if n == 0:
            l_t, b_t = series[0], series[1] - series[0]
        l_prev = l_t
        b_prev = b_t
        l_t = a * series[n] + (1 - a) * (l_prev + b_prev)
        b_t = b * (l_t - l_prev) + (1 - b)*b_prev
        
    return [l_t + i * b_t for i in range(pred_interval)]

In [126]:
def double_exponential_smoothing(series, alpha, beta):
    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # we are forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return result

In [127]:
%timeit _ = double_exponential_smoothing(ts, .2,.2)

956 µs ± 13.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [123]:
%timeit _ = des_nb(ts, .2,.2,12)

9.92 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Triple Exponential Smoothing

In [199]:
def initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen

def initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals

def triple_exponential_smoothing(series, slen, alpha, beta, gamma, n_preds):
    result = []
    seasonals = initial_seasonal_components(series, slen)

    for i in range(len(series)+n_preds):
        if i == 0: # initial values
            smooth = series[0]
            trend = initial_trend(series, slen)
            result.append(series[0])
            continue
        if i >= len(series): # we are forecasting
            m = i - len(series) + 1
            result.append((smooth + m*trend) + seasonals[i%slen])
        else:
            val = series[i]
            last_smooth, smooth = smooth, alpha*(val-seasonals[i%slen]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals[i%slen] = gamma*(val-smooth) + (1-gamma)*seasonals[i%slen]
            result.append(smooth+trend+seasonals[i%slen])
    #print(seasonals)
    return result

In [200]:
@jit(nopython=True)
def tes_nb(series, a, b, g, period, pred_interval):
    length = len(series)
    N = length // period
    l_t = 0
    b_t = 0
    c_t = np.zeros(period)
    for i in np.arange(length):
        if i == 0:
            l_t = series[0]
            b_t = np.mean(series[period:(period*2)] - series[:period]) / period
            _ = series[:N*period].reshape(N,period)
            for j in range(N):
                _[j,:] = _[j,:] / np.mean(_[j,:])
            for k in range(period):
                c_t[k] = np.mean(_[:,k])
        else:
            l_prev = l_t
            b_prev = b_t
            l_t = a * (series[i] - c_t[i%period] + (1-a)*(l_prev + b_prev))
            b_t = b * (l_t - l_prev) + (1-b)*b_prev
            c_t[i%period] = g * (series[i] - l_prev - b_prev) + (1-g)*c_t[i%period]
    
    pred_interval = np.arange(pred_interval)
    c_t = c_t[(length + pred_interval) % period]
    #print(c_t)

    return l_t + pred_interval * b_t + c_t

In [197]:
%timeit _ = tes_nb(ts, .2,.2,.2,12,12)

35.2 µs ± 6.53 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [203]:
%timeit _ = triple_exponential_smoothing(ts, 12, .2,.2,.2,12)[-12:]

2.23 ms ± 80.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
